# GBI

## 目标
通过 GBI sdk 接口完成选表和问表的能力。

## 准备工作
### 平台注册
1.先在appbuilder平台注册，获取token
2.创建BES集群，详见(https://cloud.baidu.com/doc/BES/s/0jwvyk4tv)
3.安装appbuilder-sdk

In [1]:
# !pip install appbuilder-sdk

In [14]:
import logging
import os

#  设置环境变量
os.environ["APPBUILDER_TOKEN"] = "***"


## 开发过程

### 设置表的 schema

In [3]:
SUPER_MARKET_SCHEMA = """
```
CREATE TABLE `超市营收明细表` (
  `订单编号` varchar(32) DEFAULT NULL,
  `订单日期` date DEFAULT NULL,
  `邮寄方式` varchar(32) DEFAULT NULL,
  `地区` varchar(32) DEFAULT NULL,
  `省份` varchar(32) DEFAULT NULL,
  `客户类型` varchar(32) DEFAULT NULL,
  `客户名称` varchar(32) DEFAULT NULL,
  `商品类别` varchar(32) DEFAULT NULL,
  `制造商` varchar(32) DEFAULT NULL,
  `商品名称` varchar(32) DEFAULT NULL,
  `数量` int(11) DEFAULT NULL,
  `销售额` int(11) DEFAULT NULL,
  `利润` int(11) DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4
```
"""

PRODUCT_SALES_INFO = """
现有 mysql 表 product_sales_info, 
该表的用途是: 产品收入表
```
CREATE TABLE `product_sales_info` (
  `年` int,
  `月` int,
  `产品名称` varchar,
  `收入` decimal,
  `非交付成本` decimal,
  `含交付毛利` decimal
)
```
"""

# schema 和表名的映射
SCHEMA_MAPPING = {
    "超市营收明细表": SUPER_MARKET_SCHEMA,
    "PRODUCT_SALES_INFO": PRODUCT_SALES_INFO
}

设置表的描述用于选表

In [4]:
table_descriptions = {
    "超市营收明细表": "超市营收明细表，包含超市各种信息等",
    "product_sales_info": "产品销售表"
}

### 选表

In [5]:
import appbuilder
from appbuilder.core.message import Message
from appbuilder.core.components.gbi.basic import GBISessionRecord

# 生成问表对象
select_table = appbuilder.GBISelectTable(model_name="ERNIE-Bot 4.0", table_descriptions=table_descriptions)
query = "列出超市中的所有数据"
msg = Message(query)
session = list()
select_table_result_message = select_table(message=msg, session=session)
print(f"选的表是: {select_table_result_message.content}")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
选的表是: ['超市营收明细表']


### 问表
基于上面选出的表，通过获取 shema 进行问表

In [6]:
table_schemas = [SCHEMA_MAPPING[table_name] for table_name in select_table_result_message.content]
gbi_nl2sql = appbuilder.GBINL2Sql(model_name="ERNIE-Bot 4.0", table_schemas=table_schemas)
nl2sql_result_message = gbi_nl2sql(message=msg, session=session)
print(f"sql: {nl2sql_result_message.content.sql}")
print("-----------------")
print(f"llm result: {nl2sql_result_message.content.llm_result}")

sql: 
SELECT * FROM `超市营收明细表`;
-----------------
llm result: ```sql
SELECT * FROM `超市营收明细表`;
```


设置 session

In [7]:
session.append(GBISessionRecord(query=query, answer=nl2sql_result_message.content))

再次问表

In [8]:
query2 = "查看商品类别是水果的所有数据"
msg2 = Message(query2)
nl2sql_result_message2 = gbi_nl2sql(message=msg2, session=session)
print(f"sql: {nl2sql_result_message2.content.sql}")
print("-----------------")
print(f"llm result: {nl2sql_result_message2.content.llm_result}")

sql: 
SELECT * FROM `超市营收明细表` WHERE `商品类别` = '水果'
-----------------
llm result: ```sql
SELECT * FROM `超市营收明细表` WHERE `商品类别` = '水果'
```


### 增加列选优化
实际上数据中 "商品类别" 存储的是 "新鲜水果", 那么就可以通过列选的限制来优化 sql.

In [9]:
from appbuilder.core.components.gbi.basic import ColumnItem

query2 = "查看商品类别是水果的所有数据"
msg2 = Message(query2)

column_constraint = [ColumnItem(ori_value="水果", 
                               column_name="商品类别", 
                               column_value="新鲜水果", 
                               table_name="超市营收明细表", 
                               is_like=False)]
nl2sql_result_message2 = gbi_nl2sql(message=msg2, session=session, column_constraint=column_constraint)
print(f"sql: {nl2sql_result_message2.content.sql}")
print("-----------------")
print(f"llm result: {nl2sql_result_message2.content.llm_result}")

sql: 
SELECT * FROM `超市营收明细表` WHERE `商品类别` = '新鲜水果'
-----------------
llm result: ```sql
SELECT * FROM `超市营收明细表` WHERE `商品类别` = '新鲜水果'
```


从上面我们看到，商品类别不在是 "水果" 而是 修订为 "新鲜水果"

### 增加知识优化
当计算某些特殊知识的时候，大模型是不知道的，所以需要告诉大模型具体的知识，比如:
利润率的计算方式: 利润/销售额
可以将该知识注入。具体示例如下:

In [10]:
# 注入知识
gbi_nl2sql.knowledge["利润率"] = "计算方式: 利润/销售额"

In [11]:
query3 = "列出商品类别是日用品的利润率"
msg3 = Message(query3)

nl2sql_result_message3 = gbi_nl2sql(message=msg3, session=session, column_constraint=list())
print(f"sql: {nl2sql_result_message3.content.sql}")
print("-----------------")
print(f"llm result: {nl2sql_result_message3.content.llm_result}")

sql: 
SELECT 商品类别, SUM(利润)/SUM(销售额) AS 利润率
FROM `超市营收明细表`
WHERE 商品类别 = '日用品'
GROUP BY 商品类别
-----------------
llm result: ```sql
SELECT 商品类别, SUM(利润)/SUM(销售额) AS 利润率
FROM `超市营收明细表`
WHERE 商品类别 = '日用品'
GROUP BY 商品类别
```

思考步骤：

1. 首先，我们需要从`超市营收明细表`中选择数据。
2. 根据当前问题，我们关心的是商品类别为“日用品”的数据。
3. 利润率是利润除以销售额，所以我们需要对利润和销售额进行聚合。
4. 使用`SUM`函数来计算总的利润和销售额。
5. 使用`GROUP BY`语句按商品类别进行分组，以便计算每个商品类别的利润率。
6. 最后，使用`AS`关键字给计算出的利润率命名，使其更易读。


## 调整 prompt 模版
有时候，我们希望定义自己的prompt, 选表和问表两个环节都支持 prompt 模版的定制化，但是必须遵循对应的 prompt 模版的格式。

### 选表 prompt 调整
选表的 prompt template, 必须包含 
1. {num} - 表的数量， 注意 {num} 有两个地方出现
2. {table_desc} - 表的描述
3. {query} - query, 参考下面的示例:

In [15]:
SELECT_TABLE_PROMPT_TEMPLATE = """
你是一个专业的业务人员，下面有{num}张表，具体表名如下:
{table_desc}
请根据问题帮我选择上述1-{num}种的其中相关表并返回，可以为多表，也可以为单表,
返回多张表请用“,”隔开
返回格式请参考如下示例：
问题:有多少个审核通过的投运单？
回答: ```DWD_MAT_OPERATION```
请严格参考示例只不要返回无关内容，直接给出最终答案后面的内容，分析步骤不要输出
问题:{query}
回答:
"""

In [18]:
select_table4 = appbuilder.GBISelectTable(model_name="ERNIE-Bot 4.0", 
                                          table_descriptions=table_descriptions,
                                          prompt_template=SELECT_TABLE_PROMPT_TEMPLATE)
query4 = "列出超市中的所有数据"
msg4 = Message(query4)
select_table_result_message4 = select_table4(message=msg4, session=list())
print(f"选的表是: {select_table_result_message4.content}")

选的表是: ['超市营收明细表']


## 问表 prompt 调整
问表的 prompt template 必须包含:
1. {schema} - 表的 schema 信息
2. {instrument} - 列选限制的信息
3. {kg} - 知识
4. {date} - 时间
5. {history_prompt} - 历史
6. {query} - 当前问题

参考下面的示例

In [17]:
NL2SQL_PROMPT_TEMPLATE = """
  MySql 表 Schema 如下:
  {schema}
  请根据用户当前问题，联系历史信息，仅编写1个sql，其中 sql 语句需要使用```sql ```这种 markdown 形式给出。
  请参考列选信息：
  {instrument}
  请参考知识:
  {kg}
  当前时间：{date}
  历史信息如下:
  {history_prompt}
  当前问题："{query}"
  回答：
"""

In [22]:

msg5 = Message("查看商品类别是水果的所有数据")
gbi_nl2sql5 = appbuilder.GBINL2Sql(model_name="ERNIE-Bot 4.0", table_schemas=table_schemas, prompt_template=NL2SQL_PROMPT_TEMPLATE)
nl2sql_result_message5 = gbi_nl2sql5(message=msg5, session=session)
print(f"sql: {nl2sql_result_message5.content.sql}")
print("-----------------")
print(f"llm result: {nl2sql_result_message5.content.llm_result}")

sql: 
SELECT * FROM `超市营收明细表` WHERE `商品类别` = '水果'
-----------------
llm result: ```sql
SELECT * FROM `超市营收明细表` WHERE `商品类别` = '水果'
```
